<a href="https://colab.research.google.com/github/philipp-lampert/mymandible/blob/main/data_science/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preparation
This notebook prepares the raw data for further analyses by correctly defining missing values and column types.

In [1]:
import numpy as np
import pandas as pd

We are now ready to import the dataset from the [mymandible](https://github.com/philipp-lampert/mymandible) Github repository. This is the unprocessed CSV file exported directly from the associated [RedCap](https://www.project-redcap.org/) project.

We prevent automatic detection of missing values by setting `na_filter = False` as this would replace missing values with Numpy's `np.nan` which - in contrast to Panda's newer `pd.NA` - does not allow for nullable boolean and integer columns.



In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/philipp-lampert/mymandible/main/data/preprocessing/01_raw_data.csv", na_filter = False)
df = df.replace(["NaN", ""], pd.NA)

For multiple-choice variables, RedCap exports each choice as a binary column with a naming convention of `variable___option`. Importantly, missing values are not stored directly inside each column but in an additional binary column named `variable___nan`. Therefore, we have to set each row of `variable___option` to `NaN` whenever `variable___nan == 1`.

In [5]:
nan_columns = df.filter(like = "___nan").columns
multiple_choice_variables = [name.split("___nan")[0] for name in nan_columns]

for variable in multiple_choice_variables:
    row_with_nan = df[f"{variable}___nan"] == 1
    columns = df.columns[df.columns.str.startswith(variable)]
    df.loc[row_with_nan, columns] = pd.NA
    df = df.drop(f"{variable}___nan", axis=1)

With missing values now being correctly represented in our dataframe, let's remove the auto-generated RedCap columns that are only relevant during data collecting.

In [6]:
df = df.drop(["id", "predictors_complete", "outcomes_complete", "imaging_complete"], axis = 1)

Now, we will convert each column to its appropriate datatype (boolean, integer, categorical etc.).

In [8]:
data_types = {
        "boolean": {
            "sex_female",
            "skin_transplanted",
            "flap_loss",
            "wound_infection",
            "nonunion_6_12",
            "nonunion_12_24",
        },
        "category": {
            "indication",
            "prior_flap",
            "flap_revision",
            "flap_donor_site",
            "plate_type",
            "long_plate_thickness",
            "mini_plate_thickness",
            "tmj_replacement_type",
            "flap_segment_count",
            "flap_loss_type",
            "imaging_6_12",
            "imaging_12_24_months"
        },
        "string": {
            "which_autoimmune_disease",
            "which_bleeding_disorder",
        },
        "UInt8": {"age_surgery_years", "height_cm", "weight_kg"},
        "UInt16": {"surgery_duration_min"},
        "Float32": {"bmi"},
    }

for column in df.columns:
    # All multiple-choice columns have three underscores in their name
    if "___" in column:
        df[column] = df[column].astype("boolean")
    elif column in data_types["boolean"]:
        df[column] = np.where(
            df[column] == "True",
            True,
            np.where(df[column] == "False", False, df[column]),
        )
        df[column] = df[column].astype("boolean")
    elif column.startswith("days_to_"):
        df[column] = df[column].astype("UInt16")
    else:
        for data_type in ["category", "string", "UInt8", "UInt16", "Float32"]:
            if column in data_types[data_type]:
                df[column] = df[column].astype(data_type)

Now, we will create a new derived boolean outcome variable representing if any of the most relevant complications occured (see selection in code).

In [9]:
soft_tissue_complication = [
    'complication___whd_recipient_site',
    'complication___partial_necrosis',
    'complication___bone_exposure',
    'complication_plate___exposure',
    'wound_infection'
    ]

df['soft_tissue_complication'] = np.where(df[soft_tissue_complication].isna().any(axis=1), pd.NA, df[soft_tissue_complication].any(axis=1))
df['soft_tissue_complication'] = df['soft_tissue_complication'].astype('boolean')

In [10]:
df['nonunion'] = np.where(
    (df['nonunion_6_12'].isna()) & (df['nonunion_12_24'].isna()), pd.NA,
    np.where(df[['nonunion_6_12', 'nonunion_12_24']].any(axis=1), True, False)
)
df['nonunion'] = df['nonunion'].astype('boolean')

In [11]:
any_complication = [
    'flap_loss',
    'complication___whd_donor_site',
    'complication___osteoradionecrosis',
    'complication_plate___removal',
    'complication_plate___loosening',
    'nonunion_6_12',
    'nonunion_12_24',
    'complication_bony___fracture',
    'complication_bony___dislocation'
    ]

any_complication.extend(soft_tissue_complication)

df['any_complication'] = np.where(df[any_complication].any(axis=1), True, False)
df['any_complication'] = df['any_complication'].astype('boolean')

We can now save the dataframe in the Parquet format to preserve the data types, something that would not be possible in the CSV format.

In [12]:
df.to_parquet('02_preprocessed.parquet')